<img style="float: center;" src='https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_header.png' alt="stsci_logo" width="900px"/> 

#  NIRCam Imaging - Rescale ERR extension values in data resampled to non-native pixel scales

**Authors**: B. Hilbert, based on the NIRISS imaging pipeline notebook by R. Diaz, with contributions from from Hayley Williams<br>
**Last Updated**: October 7, 2024<br>
**Pipeline Version**: 1.15.1 (Build 11.0)

**Purpose**:

This notebook shows how to make an important correction to photometric uncertainty values when
working with NIRCam Imaging data. The correction is only necessary when performing photometry
on an `i2d` file **in cases where the output image has been resampled to a non-native pixel scale**.

<div class="alert alert-block alert-warning">
    Note that default pipeline products from MAST are <b>not affected</b>, as they are always resampled
    to the native detector pixel scale. But for now, these scaling factors would need to be applied when
    doing photometry with ERR values from images resampled to different pixel scales, when running the pipeline independently.
</div>

This issue arises when trying to interpret the surface brightness units for the ERR array when
calculating the photometric uncertainty of a source. 

In the `i2d` file, the science (SCI) array, as well as the error (ERR) array, are in units of
surface brightness (MJy/sr). The output pixels effectively provide sampling points of these values
across the sky. These surface brightness values should remain invariant with output pixel scale.

However, when performing photometry on a source and using the ERR array, we see that the photometric
uncertainty values **do** change with the output pixel scale. The photometric uncertainty is calculated
by summing the ERR values in quadrature. This leads to different behavior than flux density surface
brightness (i.e. the SCI array), which remains the same regardless of pixel scale. When doing photometry,
the ERR values need to have a factor applied corresponding to the ratio of the resampled pixel scale relative to the
native pixel scale of the NIRCam detectors. For example, NIRCam SW data (native detector pixel
scale ~31.5mas) resampled to pixel scales of 30mas or 60mas need to have their ERR values scaled
by factors of (31.5/30mas) and (31.5/60mas) respectively. Similarly, NIRCam LW data (native detector
pixel scale ~63mas) resampled to pixel scales of 30mas or 60mas need to have their ERR values scaled
by factors of (63/30mas) and (63/60mas) respectively.

Discussions are in progress about possibly revising the definition of ERR values to avoid this behavior in future.


**Data**:
This example is set up to use an example dataset from
[Program ID](https://www.stsci.edu/jwst/science-execution/program-information)
2739 (PI: Pontoppidan) which is a Cycle 1 Outreach program. 
We focus on two pointings from Observation 002, in which M-16, or the
"Pillars of Creation" were observed.

Example input data to use will be downloaded as part of running this notebook.

**JWST pipeline version and CRDS context** This notebook was written for the
calibration pipeline version given above. It sets the CRDS context
to use the most recent version available in the JWST Calibration
Reference Data System (CRDS). If you use different pipeline versions or
CRDS context, please read the relevant release notes
([here for pipeline](https://github.com/spacetelescope/jwst),
[here for CRDS](https://jwst-crds.stsci.edu/)) for possibly relevant
changes.<BR>

**Recent Changes**:<br>
Oct 7, 2024: original notebook created<br>


## Table of Contents
1. [Configuration](#Configuration) 
2. [Package Imports](#Package-Imports)
3. [Convenience Functions](#Convenience-Functions)
4. [Download Data](#Download-Data)
5. [Image3 Pipeline](#Image3-Pipeline)
6. [Examine the Mosaics](#Examine-the-Mosaics)
7. [Run Photometry](#Run-Photometry)
8. [Correct the ERR values](#Correct-the-ERR-values)


## Configuration

------------------
Set basic configuration for runing notebook. 

#### Install dependencies and parameters

To make sure that the pipeline version is compatabile with the steps
discussed below and the required dependencies and packages are installed,
 you can create a fresh conda environment and install the provided
`requirements.txt` file:
```
conda create -n err_scaling python=3.11
conda activate err_scaling
pip install -r requirements.txt
```

In [1]:
# Basic import necessary for configuration
import os

### Set CRDS context and server
Before importing <code>CRDS</code> and <code>JWST</code> modules, we need
to configure our environment. This includes defining a CRDS cache
directory in which to keep the reference files that will be used by the
calibration pipeline.

If the root directory for the local CRDS cache directory has not been set
already, it will be set to create one in the home directory.

In [2]:
# ------------------------Set CRDS context and paths----------------------

# Set CRDS context (if overriding to use a specific version of reference
# files; leave commented out to use latest reference files by default)
#%env CRDS_CONTEXT  jwst_1281.pmap

# Check whether the local CRDS cache directory has been set.
# If not, set it to the user home directory
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')
# Check whether the CRDS server URL has been set.  If not, set it.
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Echo CRDS path in use
print(f"CRDS local filepath: {os.environ['CRDS_PATH']}")
print(f"CRDS file server: {os.environ['CRDS_SERVER_URL']}")

CRDS local filepath: /Users/hilbert/crds_cache
CRDS file server: https://jwst-crds.stsci.edu


## Package Imports

In [3]:
# Use the entire available screen width for this notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
# Basic system utilities for interacting with files
# ----------------------General Imports------------------------------------
import glob
from pathlib import Path

# Numpy for doing calculations
import numpy as np

# -----------------------Astroquery Imports--------------------------------
# ASCII files, and downloading demo files
from astroquery.mast import Observations

# For visualizing images
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
# %matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

# Astropy routines for visualizing detected sources:
from astropy.io import fits
from astropy.visualization import ImageNormalize, LinearStretch, LogStretch, ManualInterval

#photutils
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry
from  photutils.centroids import centroid_2dg

# for JWST calibration pipeline
import jwst
from jwst.pipeline import Image3Pipeline
import crds

# JWST pipeline utilities
from jwst import datamodels
from jwst.associations import asn_from_list  # Tools for creating association files
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base  # Definition of a Lvl3 association file

# Echo pipeline version and CRDS context in use
print(f"JWST Calibration Pipeline Version: {jwst.__version__}")
print(f"Using CRDS Context: {crds.get_context_name('jwst')}")

JWST Calibration Pipeline Version: 1.16.0


CRDS - INFO -  Calibration SW Found: jwst 1.16.0 (/Users/hilbert/miniconda3/envs/err_scaling/lib/python3.11/site-packages/jwst-1.16.0.dist-info)


Using CRDS Context: jwst_1298.pmap


## Convenience Functions

Here we define functions to streamline later actions in the notebook.

In [5]:
def measure_photometry(img, err, pixel_area_sq_arcsec, ap_radius=0.2, plot=False):
    """Given a 2D image with a single source, as well as a corresponding 2D error
    array, perform aperture photometry on the source. Return the total signal and
    error

    Parameters
    ----------
    img : numpy.ndarray
        2D array containing a single source

    err : numpy.ndarray
        2D error array associated with ``img``

    pixel_area_sq_arcsec : float
        The pixel area associated with ``img`` and ``err`` in
        units of square arcseconds

    ap_radius : float
        Aperture radius for photometry. Units are arcseconds

    plot : bool
        Create an image of the source with circular aperture and
        marked centroid location.

    Returns
    -------
    flux : float
        Photometric result. Total signal of the source
        
    flux_err : float
        Uncertainty associated with the photometric result, obtained
        using ``err``
    """    
    # Convert pixel area to steradians
    pixel_area_str = pixel_area_sq_arcsec * 2.3504e-11
    
    # Find the centroid of the star
    centroid = centroid_2dg(img, err)

    # Convert image and error arrays from MJy/sr to muJy 
    img_scaled = img * pixel_area_str * 1e12
    err_scaled = err * pixel_area_str * 1e12
    
    # Define aperture
    ap_radius_pix = ap_radius / np.sqrt(pixel_area_sq_arcsec)
    aper = CircularAperture(centroid, ap_radius_pix)
    
    # Perform photometry
    phot_table = aperture_photometry(img_scaled, aper, error=err_scaled)
    flux = phot_table['aperture_sum'].value[0]
    flux_err = phot_table['aperture_sum_err'].value[0]
    
    # Plot cutouts if desired
    if plot:
        fig,axes = plt.subplots(1, 2, figsize=(6,3), constrained_layout=True)
        
        axes[0].imshow(img, origin='lower', vmin=0, vmax=np.mean(img) + 3 * np.std(img))
        axes[1].imshow(err, origin='lower')
        
        axes[0].text(.02, .98, "SCI", fontsize=18, color='white', fontweight='bold',
                     horizontalalignment='left', verticalalignment='top', transform=axes[0].transAxes)
        axes[1].text(.02, .98, "ERR", fontsize=18, color='white', fontweight='bold',
                     horizontalalignment='left', verticalalignment='top', transform=axes[1].transAxes)
        
        axes[0].scatter(centroid[0], centroid[1], color='r', marker='+')
        axes[1].scatter(centroid[0], centroid[1], color='r', marker='+')
        
        aper_patch = Circle(centroid, ap_radius_pix, edgecolor='white', facecolor='none', lw=3)
        axes[0].add_patch(aper_patch)
        
        aper_patch = Circle(centroid, ap_radius_pix, edgecolor='white', facecolor='none', lw=3)
        axes[1].add_patch(aper_patch)
        
        axes[0].set_xticks([])
        axes[0].set_yticks([])
        axes[1].set_xticks([])
        axes[1].set_yticks([])
        
        plt.show()
        
    return flux, flux_err

In [6]:
def plot_photometry_results(fluxes, old_errors, scaled_errors):
    """Plot the results from performing photometry on the two differently-scaled mosaic images

    Parameters
    ----------
    fluxes : list
        List of fluxes from photometry. Should be one value for each of the two output pixel scales

    old_errors : list
        Uncertainties from the photometry. Should be one value for each of the two output pixel scales

    scaled_errors : list
        Uncertainties from the photometry, after scaling. Should be one value for each of the two output
        pixel scales.
    """
    fig, ax = plt.subplots()
    x_values = [0, 1, 4, 5]
    ax.scatter([0, 4], [fluxes[0]]*2, facecolor='dodgerblue', edgecolor='darkblue', lw=2, marker='o', s=150, label="30mas")
    ax.scatter([1, 5], [fluxes[1]]*2, facecolor='red', edgecolor='darkred', lw=2, marker='o', s=150, label="60mas")
    ax.errorbar([0, 1], fluxes, old_errors, color='darkblue', lw=2, capsize=5, fmt="None")
    ax.errorbar([4, 5], fluxes, scaled_errors, color='darkblue', lw=2, capsize=5, fmt="None")
        
    #finish up plot:
    ax.set_xticks([0.5, 4.5],['Before Scaling', 'After Scaling'])
    ax.tick_params(labelsize=12)
    ax.set_ylabel(r"Flux Density (${\rm \mu}$Jy)", fontsize=14)
    ax.set_xlim(-1, 6)
    
    ax.legend(fontsize=14)
    plt.tight_layout()
    plt.show()


In [7]:
def scale_err(err_value, native_scale, new_scale):
    """Scale the input ERR value by the ratio of the native to the new pixel scale.
    This is the correction to apply to the ERR extension of the i2d files in order
    to have the values behave as expected.

    Parameters
    ----------
    err_value : float
        ERR value from i2d file

    native_scale : float
        The native pixel scale for the detector. Units are arcseconds per pixel.

    new_scale : float
        New pixel scale to which the data were resampled. Units are arcseconds per pixel.

    Returns
    -------
    value : float
        Updated ``err_val``, scaled by the ratio of the pixel scales
    
    """
    return err_value * (native_scale / new_scale)

In [8]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str'):
    """Function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

## Download Data

------------------
Retrieve the calibrated data automatically from MAST using
[astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html).
MAST allows for flexibility of searching by the proposal ID and the
observation ID instead of just filenames.<br>

For illustrative purposes, we focus on data taken using the NIRCam
[F200W filter](https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-instrumentation/nircam-filters).
The files are named
`jw02739001002_02105_0000<dither>_nrca2_uncal.fits`, where *dither* refers to the
 dither step number. 
 
More information about the JWST file naming conventions can be found at:
https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/file_naming.html

In [9]:
# Set up the program information

program = '02739'
sci_observtn = "001"
    
data_dir = os.path.join('.', 'nrc_im_demo_data')
download_dir = data_dir
        
# Create directory if it does not exist
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

Identify list of example calibrated files (*_cal.fits) to download.

In [10]:
# Obtain a list of observation IDs for the specified program

sci_obs_id_table = Observations.query_criteria(instrument_name=["NIRCAM/IMAGE"],
                                               provenance_name=["CALJWST"],  # Executed observations
                                               filters=['F200W'],  # Data for Specific Filter
                                               obs_id=['jw' + program + '-o' + sci_observtn + '*']
                                               )

In [11]:
sci_obs_id_table

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str7,str12,str4,str5,str8,str4,str42,str36,float64,float64,str5,str21,int64,float64,float64,float64,float64,float64,str30,float64,str4,str2,int64,str117,str62,str63,str6,bool,float64,str9,str9
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,INFRARED,M-16,ISM; Molecular clouds; Nebulae; Protostars,jw02739-o001_t001_nircam_clear-f200w,274.7299074457347,-13.851719782834468,image,"Pontoppidan, Klaus M.",3,59805.40325079491,59824.93756190972,3221.0400000000004,1755.0,2226.0,JWST Cycle 1 Outreach Campaign,59871.55982626,2739,DD,--,POLYGON 274.693610607 -13.78846435 274.770282437 -13.790845449 274.766206195 -13.91520072 274.689493363 -13.912818346,mast:JWST/product/jw02739-o001_t001_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw02739-o001_t001_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,219392596,617131674


In [12]:
# Turn the list of visits into a list of calibrated data files

# Define types of files to select
file_dict = {'uncal': {'product_type': 'SCIENCE',
                       'productSubGroupDescription': 'CAL',
                       'calib_level': [2]}}

# Science files
sci_files_to_download = []
# Loop over visits identifying uncalibrated files that are associated
# with them
for exposure in (sci_obs_id_table):
    products = Observations.get_product_list(exposure)
    for filetype, query_dict in file_dict.items():
        filtered_products = Observations.filter_products(products, productType=query_dict['product_type'],
                                                         productSubGroupDescription=query_dict['productSubGroupDescription'],
                                                         calib_level=query_dict['calib_level'])
        sci_files_to_download.extend(filtered_products['dataURI'])

# To limit data volume, keep only files from visit 002, dithers 1 and 2, and only detector NRCA2
sw_sci_files_to_download = [fname for fname in sci_files_to_download if 'jw02739001002_02105' in fname and \
                            ('nrca2' in fname) and ('00001' in fname or '00002' in fname)]
sw_sci_files_to_download = sorted(sw_sci_files_to_download)
print(f"Science files selected for downloading: {len(sw_sci_files_to_download)}")

Science files selected for downloading: 2


In [13]:
sw_sci_files_to_download

['mast:JWST/product/jw02739001002_02105_00001_nrca2_cal.fits',
 'mast:JWST/product/jw02739001002_02105_00002_nrca2_cal.fits']

Download the files.

<div class="alert alert-block alert-warning">
Warning: If this notebook is halted during this step the downloaded file
may be incomplete, and cause crashes later on!
</div>

In [14]:
# Download the data if it does not already exist
for filename in sw_sci_files_to_download:
    sci_manifest = Observations.download_file(filename,
                                              local_path=os.path.join(data_dir, Path(filename).name))

2024-11-27 12:30:44,950 - stpipe - INFO - Found cached file ./nrc_im_demo_data/jw02739001002_02105_00001_nrca2_cal.fits with expected size 117538560.
2024-11-27 12:30:45,062 - stpipe - INFO - Found cached file ./nrc_im_demo_data/jw02739001002_02105_00002_nrca2_cal.fits with expected size 117538560.


INFO: Found cached file ./nrc_im_demo_data/jw02739001002_02105_00001_nrca2_cal.fits with expected size 117538560. [astroquery.query]
INFO: Found cached file ./nrc_im_demo_data/jw02739001002_02105_00002_nrca2_cal.fits with expected size 117538560. [astroquery.query]


## Image3 Pipeline

In the [Image3 stage of the pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html), the individual `*_cal.fits` files are combined into one single distortion corrected image. 

First, we need to create an [Association file](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/overview.html), in order to tell the pipeline which files to work on.

Find and sort all of the input files, ensuring use of absolute paths.

In [15]:
# Get a list of the *_cal.fits files
sw_sstring = os.path.join(data_dir, 'jw*nrc??_cal.fits')
sw_cal_files = sorted(glob.glob(sw_sstring))

# Expand the relative paths into absolute paths
sw_cal_files = [os.path.abspath(fname) for fname in sw_cal_files]

print(f'Found {len(sw_cal_files)} shortwave science files to process')

Found 2 shortwave science files to process


In [16]:
sw_cal_files

['/Users/hilbert/python_repos/jwst-caveat-examples/NIRCam_Imaging/nrc_im_demo_data/jw02739001002_02105_00001_nrca2_cal.fits',
 '/Users/hilbert/python_repos/jwst-caveat-examples/NIRCam_Imaging/nrc_im_demo_data/jw02739001002_02105_00002_nrca2_cal.fits']

### Create Association File

An association file lists the files to calibrate together in `Stage 3` of the pipeline.

Note that the output products (the mosaic and source catalog) will have a rootname that is specified by the `product_name` in the association file. Because of this, we'll create two association files: one for each pixel scale we are resampling to. The product name for the two cases will be different.'

<div class="alert alert-block alert-warning">
Note: The cell below may produce a warning about using full paths in association file entries. Generally, this should be ok, and the pipeline will work as expected.
</div>

In [17]:
# Create Level 3 Associations

# For the 0.03"/pixel case
product_name_0p3 = 'image3_0p3'
association = asn_from_list.asn_from_list(sw_cal_files,
                                          rule=DMS_Level3_Base,
                                          product_name=product_name_0p3)
    
association.data['asn_type'] = 'image3'
association.data['program'] = program
    
# Format association as .json file
asn_filename_0p3, serialized_0p3 = association.dump(format="json")

# Write out association file
association_im3_0p3 = os.path.join(data_dir, asn_filename_0p3)
with open(association_im3_0p3, "w") as fd:
    fd.write(serialized_0p3)

2024-11-27 12:30:55,037 - stpipe - WARNING - /Users/hilbert/miniconda3/envs/err_scaling/lib/python3.11/site-packages/jwst/associations/association.py:215: UserWarning: Input association file contains path information; note that this can complicate usage and/or sharing of such files.
  warnings.warn(err_str, UserWarning)



In [18]:
# Create a Level 3 Association for the 0.06"/pixel case
product_name_0p6 = 'image3_0p6'
association = asn_from_list.asn_from_list(sw_cal_files,
                                          rule=DMS_Level3_Base,
                                          product_name=product_name_0p6)
    
association.data['asn_type'] = 'image3'
association.data['program'] = program
    
# Format association as .json file
asn_filename_0p6, serialized_0p6 = association.dump(format="json")

# Write out association file
association_im3_0p6 = os.path.join(data_dir, asn_filename_0p6)
with open(association_im3_0p6, "w") as fd:
    fd.write(serialized_0p6)

### Run Image3 stage of the pipeline

We'll call the Image3 pipeline twice. First, we will create a final mosaic image with a pixel scale of 0.03"/pixel, which is very close to the native pixel scale of ~0.031"/pixel. The second call to the Image3 pipeline will create a final mosaic image with a pixel scale of 0.06"/pixel.

In [19]:
# Set up a dictionary for the first call to the pipeline. In this case,
# we'll resample the output onto a 0.03 arcsec/pixel grid. By setting
# the pixel_scale parameter, we can control the pixel scale of the output.

image3dict_0p3 = {}
image3dict_0p3['resample'] = {}
image3dict_0p3['resample']['pixel_scale'] = 0.03  # arcsec

In [20]:
# Set up a dictionary for the second call to the pipeline. Here,
# we'll resample the output onto a 0.06 arcsec/pixel grid.

image3dict_0p6 = {}
image3dict_0p6['resample'] = {}
image3dict_0p6['resample']['pixel_scale'] = 0.06  # arcsec

In [ ]:
# Run Stage 3. We'll call it once for each output pixel scale.
i2d_0p3 = Image3Pipeline.call(association_im3_0p3, output_dir=data_dir, steps=image3dict_0p3, save_results=True)
i2d_0p6 = Image3Pipeline.call(association_im3_0p6, output_dir=data_dir, steps=image3dict_0p6, save_results=True)

2024-11-27 12:31:05,913 - stpipe - WARNING - /Users/hilbert/miniconda3/envs/err_scaling/lib/python3.11/site-packages/jwst/associations/association.py:215: UserWarning: Input association file contains path information; note that this can complicate usage and/or sharing of such files.
  warnings.warn(err_str, UserWarning)

2024-11-27 12:31:06,638 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /Users/hilbert/crds_cache/references/jwst/nircam/jwst_nircam_pars-tweakregstep_0036.asdf
2024-11-27 12:31:06,653 - stpipe - INFO - PARS-RESAMPLESTEP parameters found: /Users/hilbert/crds_cache/references/jwst/nircam/jwst_nircam_pars-resamplestep_0001.asdf
2024-11-27 12:31:06,665 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /Users/hilbert/crds_cache/references/jwst/nircam/jwst_nircam_pars-sourcecatalogstep_0007.asdf
2024-11-27 12:31:06,693 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2024-11-27 12:31:06,694 - stpipe.Image3Pipeline.assign_mtwcs - INFO - Assign

In [ ]:
# Read in the output mosaic files
i2d_file_0p3 = os.path.join(data_dir, f'{product_name_0p3}_i2d.fits')
i2d_file_0p6 = os.path.join(data_dir, f'{product_name_0p6}_i2d.fits')

i2d_model_0p3 = datamodels.open(i2d_file_0p3)
i2d_model_0p6 = datamodels.open(i2d_file_0p6)

## Examine the Mosaics

Take a look at the resulting mosaics. Looking at the entire mosaic in these small figures, they should look identical.

In [ ]:
show_image(i2d_model_0p3.data, 0., 50, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str')

In [ ]:
show_image(i2d_model_0p6.data, 0., 50, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str')

## Run Photometry

Now, for both pixel scale mosaics, we'll pick out a single source and measure its brightness and associated error. Ideally we would see the same brightness and error regardless of pixel scale. But if the issue with performing photometry using the ERR values is present, we'll see identical brightness values but the associated error for the 0.06"/pixel case will be double that for the 0.03"/pixel case.

**Note that the first step of the photometry routine will convert the input data from units of MJy/sr to uJy, by multiplying by the resampled pixel scale and a factor of 1e12. This is separate from the scaling correction we will apply later.**

Using the coordinates below, we'll supply a cutout around our star to the photometry routine.

In [ ]:
# First the 0.03"/pixel case.
xstart_0p3, xend_0p3 = 1135, 1165
ystart_0p3, yend_0p3 = 575, 602
i2d_0p3_flux, i2d_0p3_err = measure_photometry(i2d_model_0p3.data[ystart_0p3:yend_0p3, xstart_0p3:xend_0p3],
                                               i2d_model_0p3.err[ystart_0p3:yend_0p3, xstart_0p3:xend_0p3],
                                               i2d_model_0p3.meta.photometry.pixelarea_arcsecsq,
                                               ap_radius=0.2, plot=True)

In [ ]:
# Print the photometry results. Brightness and error.
i2d_0p3_flux, i2d_0p3_err

In [ ]:
# Next, look at the same source in the 0.06"/pixel case.
# In this view, the larger pixel scale is obvious.
xstart_0p6, xend_0p6 = 567, 582
ystart_0p6, yend_0p6 = 287, 301
i2d_0p6_flux, i2d_0p6_err = measure_photometry(i2d_model_0p6.data[ystart_0p6:yend_0p6, xstart_0p6:xend_0p6],
                                               i2d_model_0p6.err[ystart_0p6:yend_0p6, xstart_0p6:xend_0p6],
                                               i2d_model_0p6.meta.photometry.pixelarea_arcsecsq,
                                               ap_radius=0.2, plot=True)

In [ ]:
# Print the photometry results for the 0.06"/pixel case.
# Compare the error value to that from the 0.03"/pixel case above.
i2d_0p6_flux, i2d_0p6_err

## Correct the ERR values 

Note how for the two pixel scales, the measured flux of the star is essentially the same, as expected, but the error is larger by about a factor of 2 in the 0.06 "/pixel image. This is due to the way the error array was defined. While the science array gives the same brightness independent of pixel scale, the error decreases with decreasing pixel scale. In order to correct this behavior, we need to scale the error values by the ratio of the detector's native pixel scale to the pixel scale of the output image. <b>This implies that outputs from the pipeline where the pixel scale has not been changed from the native pixel scale do not require this correction.</b>

In [ ]:
native_pixel_scale = np.sqrt(fits.getheader(sw_cal_files[0], 'SCI')['PIXAR_A2'])
native_pixel_scale

In [ ]:
new_0p3_err = scale_err(i2d_0p3_err, native_pixel_scale, 0.03)
new_0p6_err = scale_err(i2d_0p6_err, native_pixel_scale, 0.06)

Compare the old and new uncertainty values

In [ ]:
# Old uncertainties at 0.03"/pix and 0.06"/pix
i2d_0p3_err, i2d_0p6_err

In [ ]:
# Scaled uncertainties at 0.03"/pix and 0.06"/pix
new_0p3_err, new_0p6_err

After scaling, the ERR values match and are independent of pixel scale.

### Plot the photometry results before and after the rescaling.

In [ ]:
plot_photometry_results([i2d_0p3_flux, i2d_0p6_flux], [i2d_0p3_err, i2d_0p6_err], [new_0p3_err, new_0p6_err])

After scaling (seen on the right half of the plot), the error bar lengths are independent of the pixel scale of the mosaic image, as expected.

<img style="float: center;" src='https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_footer.png' alt="stsci_logo" width="400px"/> 